In [ ]:
#Integrantes:
#Alejandro Pacheco Barranco
#Aldo Perez Tlaxca
#Marco Antonio Hernandez Hernandez

import pandas as pd
import numpy as np

import os
for dirname, _, filenames in os.walk('/kaggle/input/kmeans/analisis.csv'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
 
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['figure.figsize'] = (16,9)
plt.style.use('ggplot')

In [ ]:
#Lectura del archivo e impresion de las primeras 5 hileras
df = pd.read_csv(r"/kaggle/input/kmeans/analisis.csv")
df.head(5)


In [ ]:
#imprime informacion estadistica que forma parte de pandas
df.describe()

In [ ]:
#Imprime la cantidad de datos que tenemos de cada una de las categorias
print(df.groupby('categoria').size())

In [ ]:
#Ahora mostraremos los datos de manera grafica
df.drop(['categoria'],1).hist()
plt.show()

In [ ]:
#Se seleccionan 3 dimensiones y se juntan en las graficas con tal de buscar correlacion
sb.pairplot(df.dropna(), hue='categoria',height=4,vars=["op","ex","ag"],kind='scatter')

In [ ]:
#Definimos X para recibir todas las entradas de una grafica 3D que imprimira la correlacion de las distintas categorias
X = np.array(df[["op","ex","ag"]])
y = np.array(df['categoria'])
X.shape

import sys
fig = plt.figure()
auto_add_to_figure=False
ax = Axes3D(fig)
fig.add_axes(ax)
colores=['blue','red','green','blue','cyan','yellow','orange','black','pink','brown','purple']
asignar=[]
for row in y:
    asignar.append(colores[row])
ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=asignar,s=60)

In [ ]:
#Esta será la grafica que usaremos para obtener k, la cual sera el punto de mayor quiebre
Nc = range(1, 20)
kmeans = [KMeans(n_clusters=i) for i in Nc]
kmeans
score = [kmeans[i].fit(X).score(X) for i in range(len(kmeans))]
score
plt.plot(Nc,score)
plt.xlabel('Numero de clusters')
plt.ylabel('Puntaje')
plt.title('Punto de Codo')
plt.show()

In [ ]:
#Ahora usaremos un promedio de clusters con tal de obtener las etiquetas y los centroids
kmeans = KMeans(n_clusters=5).fit(X)
centroids = kmeans.cluster_centers_
print(centroids)

In [ ]:
#Una vez dados los resultados, procederemos a obtener una grafica 3D de los clusters y centralizar cada uno de estos, con tal de observar diferencias

#Predecimos clusters
labels = kmeans.predict(X)
#Obtenemos los clusters centrales
C = kmeans.cluster_centers_
colores=['red','green','blue','cyan','yellow']
asignar=[]
for row in labels:
    asignar.append(colores[row])
 
fig = plt.figure()
ax = Axes3D(fig)
ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=asignar,s=60)
ax.scatter(C[:, 0], C[:, 1], C[:, 2], marker='*', c=colores, s=1000)
#Podemos ver que el Algoritmo de K-Means ha agrupado a cierto numero de usuarios, los cuales se representaron a lo largo de las 3 dimensiones que especificamos anteriormente

In [ ]:
#Haremos 3 gráficas en 2 dimensiones con las proyecciones a partir de nuestra gráfica 3D para que nos ayude a visualizar los grupos y su clasificación

#Obteniendo los valores
f1 = df['op'].values
f2 = df['ex'].values
 
plt.scatter(f1, f2, c=asignar, s=70)
plt.scatter(C[:, 0], C[:, 1], marker='*', c=colores, s=1000)
plt.show()

In [ ]:
f1 = df['op'].values
f2 = df['ag'].values
 
plt.scatter(f1, f2, c=asignar, s=70)
plt.scatter(C[:, 0], C[:, 2], marker='*', c=colores, s=1000)
plt.show()

In [ ]:
f1 = df['ex'].values
f2 = df['ag'].values
 
plt.scatter(f1, f2, c=asignar, s=70)
plt.scatter(C[:, 1], C[:, 2], marker='*', c=colores, s=1000)
plt.show()

In [ ]:
#Ahora bien, podemos ver cada uno de los clusters cuantos usuarios tiene:
copy =  pd.DataFrame()
copy['usuario']=df['usuario'].values
copy['categoria']=df['categoria'].values
copy['label'] = labels;
cantidadGrupo =  pd.DataFrame()
cantidadGrupo['color']=colores
cantidadGrupo['cantidad']=copy.groupby('label').size()
cantidadGrupo

In [ ]:
#De igual manera podemos ver la manera en la que estan divididos los clusters en rubros laborales
group_referrer_index = copy['label'] ==0
group_referrals = copy[group_referrer_index]
 
diversidadGrupo =  pd.DataFrame()
diversidadGrupo['categoria']=[0,1,2,3,4,5,6,7,8,9]
diversidadGrupo['cantidad']=group_referrals.groupby('categoria').size()
diversidadGrupo

In [ ]:
#Se buscan los clusters que esten en los puntos mas cercanos a los centroids y se mostraran
closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, X)
closest

In [ ]:
#En este caso podemos ver que en los centros tenemos una modelo,un político, una presentadora de tv,un locutor de radio y un deportista.
users=df['usuario'].values
for row in closest:
    print(users[row])

In [ ]:
#Clasificar nuevas muestras
X_new = np.array([[45.92,57.74,15.66]]) #davidguetta
 
new_labels = kmeans.predict(X_new)
print(new_labels)